# References

The RBF Neural Network is based on the implementation of [this github repo](https://github.com/darecophoenixx/wordroid.sblo.jp/tree/master/lib/keras_ex/gkernel)


Also, to train the MNIST dataset, [his example of digit recognizer](https://www.kaggle.com/wordroid/keras-rbf-layer-quick-start) is heavily used.




Links (if you cannot open):
* https://github.com/darecophoenixx/wordroid.sblo.jp/tree/master/lib/keras_ex/gkernel
* https://www.kaggle.com/wordroid/keras-rbf-layer-quick-start

# Install the Library

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

# Keras and tf versions

In [ ]:
import keras
keras.__version__

In [ ]:
import tensorflow as tf
tf.__version__

# Other libraries that are used

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

# Get MNIST data and normalize

In [ ]:
(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
X = X.reshape((X.shape[0], -1))
X_sc = X / 255.0
X_sc.shape
y_cat = to_categorical(y)

In [ ]:
X_test = X_test.reshape((X_test.shape[0], -1))
X_test_sc = X_test / 255.0


 # Choose landmarks 

In [ ]:
'''
chose landmarks from sample
pick 10 data each digit
'''
np.random.seed(0)
num_lm0 = 100
num_lm = num_lm0 * 10
init_list = []
for ii in range(10):
    init_wgt0 = X_sc[y==ii]
    init_wgt0 = init_wgt0[np.random.choice(range(init_wgt0.shape[0]), size=num_lm0, replace=False)] + \
                np.random.normal(scale=0.01, size=num_lm0*784).reshape(num_lm0, 784)
    init_list.append(init_wgt0)
init_wgt = np.vstack(init_list)
init_wgt = init_wgt[np.random.permutation(range(init_wgt.shape[0]))]
init_wgt.shape

# Construct the model

In [ ]:


np.random.seed(0)

inp = Input(shape=(28*28,), name='inp')
oup = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp)
oup = Dense(10, activation='softmax')(oup)
model = Model(inp, oup)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Train the model

In [ ]:
model.fit(X_sc, y_cat, verbose=1,batch_size=500,epochs=250)

# Predict and Analyse

In [ ]:
y_pred_test = model.predict(X_test_sc)
y_pred_train = model.predict(X_sc)

In [ ]:
print("Training accuracy: ",accuracy_score(y,np.argmax(y_pred_train,axis=1)))
print("Test accuracy: ",accuracy_score(y_test,np.argmax(y_pred_test,axis=1)))

In [ ]:
print("Conf matrix train:\n ",confusion_matrix(y,np.argmax(y_pred_train,axis=1)))
print("Conf matrix test:\n ",confusion_matrix(y_test,np.argmax(y_pred_test,axis=1)))

In [ ]:
print("PRFS train:\n",precision_recall_fscore_support(y,np.argmax(y_pred_train,axis=1)))
print("PRFS test:\n",precision_recall_fscore_support(y_test,np.argmax(y_pred_test,axis=1)))

# Save and Load the Model

In [ ]:
model.save_weights("rbfnn.h5")

In [ ]:
inp2 = Input(shape=(28*28,), name='inp')
oup2 = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp2)
oup2 = Dense(10, activation='softmax')(oup2)
model2 = Model(inp2, oup2)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.load_weights("rbfnn.h5")

In [ ]:
y_pred_test = model2.predict(X_test_sc)
y_pred_train = model2.predict(X_sc)

In [ ]:
print("Training accuracy: ",accuracy_score(y,np.argmax(y_pred_train,axis=1)))
print("Test accuracy: ",accuracy_score(y_test,np.argmax(y_pred_test,axis=1)))